In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Business Cases for Data Science
## Business Case 5 -  Retail Challenge

### Group AA
**Members**:
- Emil Ahmadov (m20201004@novaims.unl.pt)
- Doris Macean (m20200609@novaims.unl.pt)
- Doyun Shin (m20200565@novaims.unl.pt)
- Anastasiia Tagiltseva (m20200041@novaims.unl.pt)

<a class="anchor" id="0.1"></a>

# **Table of Contents**

1. [Business Understanding](#1)

2. [Data Understanding](#2)

3. [Data Preparation](#3)
  
4. [Exploratory Data Analysis-Quarteraly Analysis](#4)
      
5. [PoS Clusters](#5)
      
6. [Forecast](#6)

# **1. Business Understanding** <a class="anchor" id="1"></a>

- 21 Families of Products  
- 178 Categories of Products
- 1 535 Brands Products
- 8 660 SKUs
- 410 Point-of-Sales
- 1 Jan 2016 -> 1 Nov 2019
- 18.6 Gb of data

**Challenges**

1. Data Analysis and Transformation and some Data/Feature Engineering
2. How can I understand each Point-of-Sale characteristics ?
Quarterly analysis of
Top products sold
Market Share (Family, Category), preferences
Product co-ocorrences
3. Point-of-Sales Clustering
Value
Product preference
4. Units Product forecast (6 weeks ahead)
Units Product forecast by Poit-of-Sale (6 weeks ahead

### Project Plan
| Phase | Time | Resources | Risks |
| :--: | :--------: |:--: | :--------: |
| Business Understanding | 2 days | All analysts | Economic and market changes |
| Data Understanding | 2 days | All analysts | Data problems, technological problems |
| Data Preparation | 2 days | Data scientists, DB engineers | Data problems, technological problems |
| Modeling |4 days | Data scientists | Technological problems, inability to build adequate model |
| Evaluation | 2 days | All analysts | Economic change inability to implement results |
| Deployment | 2 days | Data scientists, DB engineers, implementation team | Economic change inability to implement results |

# **2. Data Understanding** <a class="anchor" id="2"></a>

### Metadata
| Name | Meaning | 
| :--: | :--------|
| ProductFamily_ID | Families of Products|
| ProductCategory_ID | Categories of Products|
| ProductBrand_ID | Brands Products|
| ProductName_ID | Product Name|
| ProductPackSKU_ID | SKU|
| Point-of-Sale_ID | Point-of-Sales|
| Date | Date of Sales|
| Measures | Sell-out units/Sell-out values|
| Value | Value of measure|

# **3. Data Preparation** <a class="anchor" id="3"></a>

In [2]:
import sys
import os
import io

import pandas as pd
import numpy as np
import pickle

# Apriori & Recommendation
import mlxtend as ml
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from sklearn.metrics.pairwise import cosine_similarity

from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Datetime handling
import datetime

# Visualization & Clustering 1
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# Visualization & Clustering 2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import seaborn as sns
color = sns.color_palette()
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, text
import networkx as nx
import plotly.express as px
from sklearn.cluster import KMeans, AgglomerativeClustering
import plotly.graph_objs as go
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt



# Forecasting
import pmdarima as pm

#Only Applicable for Google Cloud Platform usage.
#from google.cloud import storage
#from io import StringIO
#from io import BytesIO   

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
#Only Applicable for Google Cloud Platform usage.
#client = storage.Client()
#bucket = client.get_bucket('sdymain')
#blob = bucket.get_blob(f"BC5Data_Quantity.pkl")
#bt = blob.download_as_bytes()
#data = pickle.load(BytesIO(bt))

In [4]:
data = pd.read_pickle('BC5Data_Quantity.pkl')

In [5]:
data.head()

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  ProductName_ID  \
0                 1                  33               35              77   
1                 1                  33              313             657   
2                 1                  33              313             657   
3                 1                  33              313             657   
4                 1                  33              313             657   

        Date  Point-of-Sale_ID  Quantity  
0 2016-12-27               150         1  
1 2016-01-01                13         1  
2 2016-01-01                35         1  
3 2016-01-01                77         1  
4 2016-01-01               104         1

In [6]:
data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77529406 entries, 0 to 77529405
Data columns (total 7 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ProductFamily_ID    uint64        
 1   ProductCategory_ID  uint64        
 2   ProductBrand_ID     uint64        
 3   ProductName_ID      uint64        
 4   Date                datetime64[ns]
 5   Point-of-Sale_ID    uint64        
 6   Quantity            int32         
dtypes: datetime64[ns](1), int32(1), uint64(5)
memory usage: 3.8 GB


In [7]:
a = data['Point-of-Sale_ID'].unique()
a.sort()
print(a)

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 245 246 24

In [8]:
len(data['Point-of-Sale_ID'].unique()) # Result with above code shows there is no missing PoS number from PoS1 to PoS410

410

In [9]:
data.min()

ProductFamily_ID                        1
ProductCategory_ID                      1
ProductBrand_ID                         1
ProductName_ID                          1
Date                  2016-01-01 00:00:00
Point-of-Sale_ID                        1
Quantity                                1
dtype: object

In [10]:
data.max()

ProductFamily_ID                       21
ProductCategory_ID                    178
ProductBrand_ID                      1535
ProductName_ID                       2850
Date                  2019-11-01 00:00:00
Point-of-Sale_ID                      410
Quantity                              287
dtype: object

In [11]:
# Categorical values are also stored as integers temporarily
data['ProductFamily_ID'] = data['ProductFamily_ID'].astype('uint8')
data['ProductCategory_ID'] = data['ProductCategory_ID'].astype('uint8')
data['ProductBrand_ID'] = data['ProductBrand_ID'].astype('uint16')
data['ProductName_ID'] = data['ProductName_ID'].astype('uint16')
data['Point-of-Sale_ID'] = data['Point-of-Sale_ID'].astype('uint16')
data['Quantity'] = data['Quantity'].astype('uint16')

In [12]:
data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77529406 entries, 0 to 77529405
Data columns (total 7 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ProductFamily_ID    uint8         
 1   ProductCategory_ID  uint8         
 2   ProductBrand_ID     uint16        
 3   ProductName_ID      uint16        
 4   Date                datetime64[ns]
 5   Point-of-Sale_ID    uint16        
 6   Quantity            uint16        
dtypes: datetime64[ns](1), uint16(4), uint8(2)
memory usage: 1.3 GB


In [13]:
counts = data['ProductFamily_ID'].value_counts().sort_values(ascending=False)[:15]

fig = px.bar(x = list(map(str, counts.index)), y = counts.values, labels={'x':'Family ID', 'y':'Count'})
fig.update_layout(showlegend = False)
fig.show()

In [14]:
counts = data['ProductCategory_ID'].value_counts().sort_values(ascending=False)
temp = counts[:15]
temp['Other'] = counts[10:].sum()
counts = temp

fig = px.pie(names = counts.index, values = counts.values, title = 'Product Categories Distribution')
fig.show()

In [15]:
counts = data['ProductBrand_ID'].value_counts().sort_values(ascending=False)[:10]

fig = px.bar(x = list(map(str, counts.index)), y = counts.values, labels={'x':'Brand ID', 'y':'Count'})
fig.update_layout(showlegend = False)
fig.show()

## 3.2 Feature Engineering

In [16]:
%%time
## year month quarter week
data['week'] = pd.DatetimeIndex(data['Date']).week
data['month'] = pd.DatetimeIndex(data['Date']).month
data['quarter'] = pd.DatetimeIndex(data['Date']).quarter
data['year'] = pd.DatetimeIndex(data['Date']).year

CPU times: user 21.3 s, sys: 1.23 s, total: 22.6 s
Wall time: 22.6 s


In [17]:
data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77529406 entries, 0 to 77529405
Data columns (total 11 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ProductFamily_ID    uint8         
 1   ProductCategory_ID  uint8         
 2   ProductBrand_ID     uint16        
 3   ProductName_ID      uint16        
 4   Date                datetime64[ns]
 5   Point-of-Sale_ID    uint16        
 6   Quantity            uint16        
 7   week                int64         
 8   month               int64         
 9   quarter             int64         
 10  year                int64         
dtypes: datetime64[ns](1), int64(4), uint16(4), uint8(2)
memory usage: 3.6 GB


In [18]:
data.max()

ProductFamily_ID                       21
ProductCategory_ID                    178
ProductBrand_ID                      1535
ProductName_ID                       2850
Date                  2019-11-01 00:00:00
Point-of-Sale_ID                      410
Quantity                              287
week                                   53
month                                  12
quarter                                 4
year                                 2019
dtype: object

In [19]:
data.min()
#Check Value whether there is actually negative value

ProductFamily_ID                        1
ProductCategory_ID                      1
ProductBrand_ID                         1
ProductName_ID                          1
Date                  2016-01-01 00:00:00
Point-of-Sale_ID                        1
Quantity                                1
week                                    1
month                                   1
quarter                                 1
year                                 2016
dtype: object

In [20]:
data['week'] = data['week'].astype('int8')
data['month'] = data['month'].astype('int8')
data['quarter'] = data['quarter'].astype('int8')
data['year'] = data['year'] - 2000
data['year'] = data['year'].astype('int8')

In [21]:
data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77529406 entries, 0 to 77529405
Data columns (total 11 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ProductFamily_ID    uint8         
 1   ProductCategory_ID  uint8         
 2   ProductBrand_ID     uint16        
 3   ProductName_ID      uint16        
 4   Date                datetime64[ns]
 5   Point-of-Sale_ID    uint16        
 6   Quantity            uint16        
 7   week                int8          
 8   month               int8          
 9   quarter             int8          
 10  year                int8          
dtypes: datetime64[ns](1), int8(4), uint16(4), uint8(2)
memory usage: 1.6 GB


# 4. Exploratory Data Analysis-Quarteraly Analysis <a class="anchor" id="4"></a>

In [22]:
counts = data[['Date', 'Quantity']].groupby('Date').count().groupby(pd.Grouper(freq='Q')).sum()
fig = px.scatter(counts['Quantity'], trendline='lowess', labels = {'Quantity':'Count Of Sales', 'Date':'Date By Quarter'}, title = 'Quarterly Analisys of Sales Amount')
fig.update_layout(showlegend = False)
fig.show() 

## 4.1 Top Products

### 4.1.1 Top Family

In [23]:
family = data[['ProductFamily_ID','Point-of-Sale_ID','year','quarter','Quantity']].groupby(['ProductFamily_ID','Point-of-Sale_ID','year','quarter']).sum().reset_index()
family.head()

ProductFamily_ID  Point-of-Sale_ID  year  quarter  Quantity
0                 1                 1    16        1      6428
1                 1                 1    16        2      6504
2                 1                 1    16        3      7155
3                 1                 1    16        4      6521
4                 1                 1    17        1      6296

In [24]:
check1 = pd.DataFrame()
for i in family["Point-of-Sale_ID"].unique():
    for q in family["quarter"].unique():
        for y in family["year"].unique():
            check = family[(family["Point-of-Sale_ID"]==i)&(family["quarter"]==q)&(family["year"]==y)]
            check1 = check1.append(check.sort_values(by=["Quantity"],ascending=False).head(5))
check1.sort_values(by=['Point-of-Sale_ID',"year","quarter"], inplace= True)

In [25]:
# Is there 5 families per PoS?
# 4 years * 4 quarters * 5 families = 80, value count for each PoS should be 80.
counts = check1['Point-of-Sale_ID'].value_counts()
counts[counts !=80]

96    65
Name: Point-of-Sale_ID, dtype: int64

In [26]:
del counts
pd.set_option('display.max_rows', 65)
check1[check1['Point-of-Sale_ID']==96]

ProductFamily_ID  Point-of-Sale_ID  year  quarter  Quantity
53962                  9                96    16        1      4224
73628                 12                96    16        1      3535
1520                   1                96    16        1      3331
86742                 14                96    16        1      1564
93299                 15                96    16        1      1535
53963                  9                96    16        2      4496
73629                 12                96    16        2      3880
1521                   1                96    16        2      3189
128622                21                96    16        2      1696
8078                   2                96    16        2      1238
53964                  9                96    16        3      3957
73630                 12                96    16        3      3370
1522                   1                96    16        3      2864
128623                21                96    16        3      1570
8079                   2                96    16        3      1166
53965                  9                96    16        4      3786
73631                 12                96    16        4      3260
1523                   1                96    16        4      2800
86745                 14                96    16        4      1588
93302                 15                96    16        4      1409
53966                  9                96    17        1      3808
73632                 12                96    17        1      3203
1524                   1                96    17        1      2665
128625                21                96    17        1      1242
93303                 15                96    17        1      1221
53967                  9                96    17        2      3907
73633                 12                96    17        2      3275
1525                   1                96    17        2      2543
128626                21                96    17        2      1672
8082                   2                96    17        2      1074
53968                  9                96    17        3      3822
73634                 12                96    17        3      3234
1526                   1                96    17        3      2664
128627                21                96    17        3      1373
8083                   2                96    17        3      1094
53969                  9                96    17        4      3810
73635                 12                96    17        4      3321
1527                   1                96    17        4      2713
86749                 14                96    17        4      1586
128628                21                96    17        4      1425
53970                  9                96    18        1      3859
73636                 12                96    18        1      3337
1528                   1                96    18        1      2639
86750                 14                96    18        1      1409
93307                 15                96    18        1      1341
53971                  9                96    18        2      3947
73637                 12                96    18        2      3636
1529                   1                96    18        2      2577
128630                21                96    18        2      1699
8086                   2                96    18        2      1117
53972                  9                96    18        3      3903
73638                 12                96    18        3      3515
1530                   1                96    18        3      2588
128631                21                96    18        3      1505
8087                   2                96    18        3      1133
53973                  9                96    18        4      4133
73639                 12                96    18        4      3788
1531                   1                96    18        4      2

In [27]:
# No Sales record after 2019 Q1 for PoS 96. We assume this store is permanently closed, We still keep in our data for reference, but will drop it when necessary, for other analyses.
fam_pos_q_top5 = check1.copy()
del check1

In [28]:
fam_pos_q_top5.to_csv (r'fam_pos_q_top5.csv', index = False, header=True)

In [29]:
test = data[['ProductFamily_ID','ProductBrand_ID','Point-of-Sale_ID','year','quarter','Quantity']].groupby(['ProductFamily_ID','ProductBrand_ID','Point-of-Sale_ID','year','quarter']).sum().reset_index()

# For Brand level breakdown, we concat 
# Alternative method would be adding a new loop line in the next cell below "for y in test['year'].unique() another loop based on Category, and add a new condition to check object, for better presentation, but it takes very long to process the loops."
test = test[test['ProductFamily_ID'].isin(fam_pos_q_top5['ProductFamily_ID'].unique())]
test['F_Brand'] = test['ProductFamily_ID'].astype(str).str.cat(test['ProductBrand_ID'].astype(str), sep='_')
test = test.drop(['ProductFamily_ID','ProductBrand_ID'],axis = 1)
test['B_Quantity'] = test['Quantity']
test = test.drop(['Quantity'],axis = 1)
test.head()

Point-of-Sale_ID  year  quarter F_Brand  B_Quantity
0               150    16        4    1_35           1
1               163    17        2    1_35           1
2                 1    16        1    1_72           8
3                 1    16        2    1_72          14
4                 1    16        3    1_72           8

In [30]:
fam_brand_pos_q_top5 = pd.DataFrame()
for i in test["Point-of-Sale_ID"].unique():
    for q in test["quarter"].unique():
        for y in test["year"].unique():
            check = test[(test["Point-of-Sale_ID"]==i)&(test["quarter"]==q)&(test["year"]==y)]
            fam_brand_pos_q_top5 = fam_brand_pos_q_top5.append(check.sort_values(by=["B_Quantity"],ascending=False).head(5))
fam_brand_pos_q_top5.sort_values(by=['Point-of-Sale_ID',"year","quarter"], inplace= True)

In [31]:
fam_brand_pos_q_top5[['Family','Brand']] = fam_brand_pos_q_top5['F_Brand'].str.split('_',expand=True)
fam_brand_pos_q_top5 = fam_brand_pos_q_top5.drop('F_Brand',axis = 1)
fam_brand_pos_q_top5.head()

Point-of-Sale_ID  year  quarter  B_Quantity Family Brand
2705573                 1    16        1        1143     18   940
1406310                 1    16        1         978      9  1305
361531                  1    16        1         845      1   933
2532398                 1    16        1         682     15   487
2400060                 1    16        1         679     14   226

In [32]:
fam_brand_pos_q_top5.to_csv ('cat_brand_pos_q_top5.csv')
del fam_brand_pos_q_top5
del test

### 4.1.2 Top Category

In [33]:
category = data[['ProductCategory_ID','Point-of-Sale_ID','year','quarter','Quantity']].groupby(['ProductCategory_ID','Point-of-Sale_ID','year','quarter']).sum().reset_index()
category.head()

ProductCategory_ID  Point-of-Sale_ID  year  quarter  Quantity
0                   1                 1    16        1     387.0
1                   1                 1    16        2     267.0
2                   1                 1    16        3     207.0
3                   1                 1    16        4     363.0
4                   1                 1    17        1     393.0

In [34]:
# Check Quantity to convert to int
category.max()

ProductCategory_ID      178.0
Point-of-Sale_ID        410.0
year                     19.0
quarter                   4.0
Quantity              67452.0
dtype: float64

In [35]:
# Check Quantity to convert to int
# Check presence of Quantity = 0
category.min()

ProductCategory_ID     1.0
Point-of-Sale_ID       1.0
year                  16.0
quarter                1.0
Quantity               1.0
dtype: float64

In [36]:
# Check quantity to convert to int
test = category[category['Quantity'] % 1 != 0]
test

Empty DataFrame
Columns: [ProductCategory_ID, Point-of-Sale_ID, year, quarter, Quantity]
Index: []

In [37]:
category.Quantity = category.Quantity.astype('int32')

In [38]:
#category = category[category['Point-of-Sale_ID'] != 96]

In [39]:
cat_pos_q_top5 = pd.DataFrame()
for i in category["Point-of-Sale_ID"].unique():
    for q in category["quarter"].unique():
        for y in category["year"].unique():
            check = category[(category["Point-of-Sale_ID"]==i)&(category["quarter"]==q)&(category["year"]==y)]
            cat_pos_q_top5 = cat_pos_q_top5.append(check.sort_values(by=["Quantity"],ascending=False).head(5))
cat_pos_q_top5.sort_values(by=['Point-of-Sale_ID',"year","quarter"], inplace= True)

cat_pos_q_top5.head()

ProductCategory_ID  Point-of-Sale_ID  year  quarter  Quantity
585861                 178                 1    16        1     25855
577886                 175                 1    16        1      3027
120662                  27                 1    16        1      1818
13114                    3                 1    16        1      1570
96489                   22                 1    16        1       823

In [40]:
cat_pos_q_top5.to_csv (r'cat_pos_q_top5.csv', index = False, header=True)

In [41]:
cat_pos_q_top5['ProductCategory_ID'].unique()

array([178, 175,  27,   3,  22,   2,  11,  28,  34,   1, 157, 109, 172,
       149, 161, 127, 133,  10, 115,  13, 132,  39], dtype=uint64)

In [42]:
test = data[['ProductCategory_ID','ProductBrand_ID','Point-of-Sale_ID','year','quarter','Quantity']].groupby(['ProductCategory_ID','ProductBrand_ID','Point-of-Sale_ID','year','quarter']).sum().reset_index()

# For Brand level breakdown, we concat 
# Alternative method would be adding a new loop line in the next cell below "for y in test['year'].unique() another loop based on Category, and add a new condition to check object, for better presentation, but it takes very long to process the loops."
test = test[test['ProductCategory_ID'].isin(cat_pos_q_top5['ProductCategory_ID'].unique())]
test['C_Brand'] = test['ProductCategory_ID'].astype(str).str.cat(test['ProductBrand_ID'].astype(str), sep='_')
test = test.drop(['ProductCategory_ID','ProductBrand_ID'],axis = 1)
test['B_Quantity'] = test['Quantity']
test = test.drop(['Quantity'],axis = 1)
test.head()

Point-of-Sale_ID  year  quarter C_Brand  B_Quantity
0                 1    16        1    1_45           6
1                 1    16        2    1_45           8
2                 1    16        3    1_45           2
3                 1    16        4    1_45           6
4                 1    17        1    1_45           6

In [43]:
cat_brand_pos_q_top5 = pd.DataFrame()
for i in test["Point-of-Sale_ID"].unique():
    for q in test["quarter"].unique():
        for y in test["year"].unique():
            check = test[(test["Point-of-Sale_ID"]==i)&(test["quarter"]==q)&(test["year"]==y)]
            cat_brand_pos_q_top5 = cat_brand_pos_q_top5.append(check.sort_values(by=["B_Quantity"],ascending=False).head(5))
cat_brand_pos_q_top5.sort_values(by=['Point-of-Sale_ID',"year","quarter"], inplace= True)

In [44]:
cat_brand_pos_q_top5[['Category','Brand']] = cat_brand_pos_q_top5['C_Brand'].str.split('_',expand=True)
cat_brand_pos_q_top5 = cat_brand_pos_q_top5.drop('C_Brand',axis = 1)
cat_brand_pos_q_top5.head()

Point-of-Sale_ID  year  quarter  B_Quantity Category Brand
2162295                 1    16        1        1143      175   940
4189060                 1    16        1         978      178  1305
3419902                 1    16        1         845      178   933
575440                  1    16        1         679       27   226
604871                  1    16        1         669       28   574

In [45]:
cat_brand_pos_q_top5.to_csv ('cat_brand_pos_q_top5.csv')
del cat_brand_pos_q_top5
del test

### 4.1.3 Top Product (name)

In [46]:
name = data[['ProductName_ID','Point-of-Sale_ID','year','quarter','Quantity']].groupby(['ProductName_ID','Point-of-Sale_ID','year','quarter']).sum().reset_index()
name.head()

ProductName_ID  Point-of-Sale_ID  year  quarter  Quantity
0               1                 9    19        2         4
1               1                 9    19        3         8
2               1                17    19        2         8
3               1                17    19        3        32
4               1                17    19        4         8

In [47]:
#name = name[name['Point-of-Sale_ID'] != 96]

In [48]:
name_pos_q_top5 = pd.DataFrame()
for i in name["Point-of-Sale_ID"].unique():
    for q in name["quarter"].unique():
        for y in name["year"].unique():
            check = name[(name["Point-of-Sale_ID"]==i)&(name["quarter"]==q)&(name["year"]==y)]
            name_pos_q_top5 = name_pos_q_top5.append(check.sort_values(by=["Quantity"],ascending=False).head(5))
name_pos_q_top5.sort_values(by=['Point-of-Sale_ID',"year","quarter"], inplace= True)

name_pos_q_top5.head()

ProductName_ID  Point-of-Sale_ID  year  quarter  Quantity
3800815            2412                 1    16        1       978
2699195            1766                 1    16        1       845
689241              481                 1    16        1       679
1734434            1147                 1    16        1       669
1838121            1234                 1    16        1       663

In [49]:
name_pos_q_top5.to_csv (r'name_pos_q_top5.csv', index = False, header=True)

## 4.2 Market Share (Quantity) and Preferences at Family and Category Level

In [50]:
# Total units of all items sold for each PoS
total_units_pos = family[['Point-of-Sale_ID','year','quarter','Quantity']].groupby(['Point-of-Sale_ID','year','quarter']).sum().reset_index()
total_units_pos.rename(columns={'Quantity': 'PoSTotalSales'}, inplace = True)
total_units_pos.head()

Point-of-Sale_ID  year  quarter  PoSTotalSales
0                 1    16        1        38477.0
1                 1    16        2        37515.0
2                 1    16        3        39340.0
3                 1    16        4        40121.0
4                 1    17        1        38012.0

### 4.2.2 Market Share Family Level

In [51]:
fam_ms = pd.merge(family, total_units_pos, how='left', left_on=['Point-of-Sale_ID','year','quarter'], right_on = ['Point-of-Sale_ID','year','quarter'])
fam_ms.head()

ProductFamily_ID  Point-of-Sale_ID  year  quarter  Quantity  PoSTotalSales
0                 1                 1    16        1      6428        38477.0
1                 1                 1    16        2      6504        37515.0
2                 1                 1    16        3      7155        39340.0
3                 1                 1    16        4      6521        40121.0
4                 1                 1    17        1      6296        38012.0

In [52]:
#check
print(len(fam_ms))
print(len(family))

133658
133658


In [53]:
# For all
fam_ms['MarketShare'] = fam_ms['Quantity'] / fam_ms['PoSTotalSales']
fam_ms['MarketShare'] = fam_ms['MarketShare'].round(decimals=4)
fam_ms.head()

ProductFamily_ID  Point-of-Sale_ID  year  quarter  Quantity  PoSTotalSales  \
0                 1                 1    16        1      6428        38477.0   
1                 1                 1    16        2      6504        37515.0   
2                 1                 1    16        3      7155        39340.0   
3                 1                 1    16        4      6521        40121.0   
4                 1                 1    17        1      6296        38012.0   

   MarketShare  
0       0.1671  
1       0.1734  
2       0.1819  
3       0.1625  
4       0.1656

In [54]:
# For top 5
fam_ms_top = pd.merge(fam_pos_q_top5, fam_ms.loc[:, fam_ms.columns != 'Quantity'], how='left', left_on=['ProductFamily_ID','Point-of-Sale_ID','year','quarter'], right_on = ['ProductFamily_ID','Point-of-Sale_ID','year','quarter'])
fam_ms_top.head()

ProductFamily_ID  Point-of-Sale_ID  year  quarter  Quantity  PoSTotalSales  \
0                 9                 1    16        1      8877        38477.0   
1                 1                 1    16        1      6428        38477.0   
2                12                 1    16        1      6019        38477.0   
3                15                 1    16        1      3090        38477.0   
4                18                 1    16        1      3029        38477.0   

   MarketShare  
0       0.2307  
1       0.1671  
2       0.1564  
3       0.0803  
4       0.0787

In [55]:
#check
print(len(fam_pos_q_top5))
print(len(fam_ms_top))

32785
32785


In [56]:
# Check for PoS and quarters that heavily rely on one Product Family >> specialization/diversification choices
fam_ms_top[fam_ms_top['MarketShare']>0.4].sort_values('MarketShare',ascending=False)

ProductFamily_ID  Point-of-Sale_ID  year  quarter  Quantity  \
24810                21               311    17        2     13789   
24850                21               311    19        2     14900   
24855                21               311    19        3     13468   
24815                21               311    17        3     12365   
24835                21               311    18        3     12533   
24795                21               311    16        3     11064   
24830                21               311    18        2     12981   
24790                21               311    16        2     10063   

       PoSTotalSales  MarketShare  
24810        27637.0       0.4989  
24850        31011.0       0.4805  
24855        28796.0       0.4677  
24815        26529.0       0.4661  
24835        27111.0       0.4623  
24795        24458.0       0.4524  
24830        29051.0       0.4468  
24790        24610.0       0.4089

In [57]:
fam_ms_top.to_csv (r'fam_ms_top.csv', index = False, header=True)

### 4.2.3 Market Share Category Level

In [58]:
cat_ms = pd.merge(category, total_units_pos, how='left', left_on=['Point-of-Sale_ID','year','quarter'], right_on = ['Point-of-Sale_ID','year','quarter'])
cat_ms.head()

ProductCategory_ID  Point-of-Sale_ID  year  quarter  Quantity  \
0                   1                 1    16        1       387   
1                   1                 1    16        2       267   
2                   1                 1    16        3       207   
3                   1                 1    16        4       363   
4                   1                 1    17        1       393   

   PoSTotalSales  
0        38477.0  
1        37515.0  
2        39340.0  
3        40121.0  
4        38012.0

In [59]:
#check
print(len(cat_ms))
print(len(category))

592418
592418


In [60]:
# For all
cat_ms['MarketShare'] = cat_ms['Quantity'] / cat_ms['PoSTotalSales']
cat_ms['MarketShare'] = cat_ms['MarketShare'].round(decimals=4)
cat_ms.head()

ProductCategory_ID  Point-of-Sale_ID  year  quarter  Quantity  \
0                   1                 1    16        1       387   
1                   1                 1    16        2       267   
2                   1                 1    16        3       207   
3                   1                 1    16        4       363   
4                   1                 1    17        1       393   

   PoSTotalSales  MarketShare  
0        38477.0       0.0101  
1        37515.0       0.0071  
2        39340.0       0.0053  
3        40121.0       0.0090  
4        38012.0       0.0103

In [61]:
# For top 5
cat_ms_top = pd.merge(cat_pos_q_top5, cat_ms.loc[:, cat_ms.columns != 'Quantity'], how='left', left_on=['ProductCategory_ID','Point-of-Sale_ID','year','quarter'], right_on = ['ProductCategory_ID','Point-of-Sale_ID','year','quarter'])
cat_ms_top.head()

ProductCategory_ID  Point-of-Sale_ID  year  quarter  Quantity  \
0                 178                 1    16        1     25855   
1                 175                 1    16        1      3027   
2                  27                 1    16        1      1818   
3                   3                 1    16        1      1570   
4                  22                 1    16        1       823   

   PoSTotalSales  MarketShare  
0        38477.0       0.6720  
1        38477.0       0.0787  
2        38477.0       0.0472  
3        38477.0       0.0408  
4        38477.0       0.0214

In [62]:
#check
print(len(cat_pos_q_top5))
print(len(cat_ms_top))

32785
32785


In [63]:
# Check for PoS and quarters that heavily rely on one Product category >> specialization/diversification choices
# note: as a category can belong to multiple families, the market share value can be much higher.
cat_ms_top[cat_ms_top['MarketShare']>0.4].sort_values('MarketShare',ascending=False)

ProductCategory_ID  Point-of-Sale_ID  year  quarter  Quantity  \
18675                 178               234    18        3     12293   
230                   178                 3    19        3     34777   
18615                 178               233    19        3     21448   
18595                 178               233    18        3     20268   
6470                  178                81    19        3     17154   
...                   ...               ...   ...      ...       ...   
31065                 178               389    18        1     23183   
25465                 178               319    18        1      5643   
25460                 178               319    17        4      5390   
25480                 178               319    18        4      5832   
25485                 178               319    19        1      5826   

       PoSTotalSales  MarketShare  
18675        14306.0       0.8593  
230          40561.0       0.8574  
18615        25035.0       0.8567  
18595        23713.0       0.8547  
6470         20082.0       0.8542  
...              ...          ...  
31065        50773.0       0.4566  
25465        12414.0       0.4546  
25460        11883.0       0.4536  
25480        13042.0       0.4472  
25485        13107.0       0.4445  

[6541 rows x 7 columns]

In [64]:
cat_ms_top[cat_ms_top['MarketShare']>0.4]['ProductCategory_ID'].unique()

array([178], dtype=uint64)

In [65]:
cat_ms_top.to_csv (r'cat_ms_top.csv', index = False, header=True)

**Consideration:**
- ProductCategory_178 is by far the most dominant source of revenue for our client

## 4.3 Product Co-occurrences per PoS per Quarter

### 4.3.1 Co-occurrence by Product

In [66]:
# This function would allow for a highly customizable Market Bastket Analysis.
# It doesn't just check for product cooccurrence, but also allows for potential substitutes and complementary products.
# However, it requires very high computational power, especially when looking at  which would be accessible at an enterprise level. Hence, we leave this function in case it is feasible to implement it to the business during the deploymnent stage.

# pos = PoSnumber
# yyyy = year
# q = Quarter

# ms = apriori min_support value, 0.05 by default
# cmt = confidence min_threshold, 0.5 by default
# lmt = lift min_threshold, 1.5 by default

# cmptc = complement: min_threshold for confidence
# cmptl = complement: min_threshold for lift
# cmptl_c = complement: ceiling for lift (e.g, cmptl_c=1.5, max lift value is 1.5)
#subsmt = substitutes: min_threshold for lift
def quarterly_mba(pos, yyyy=2016, q = 1, ms=0.05, cmt=0.50, lmt=1.5, cmptc=0.8, cmptl=1.8, cmptl_c=99, subsmt=0.0):
    cmptl_c=cmptl_c
    rConf_list=[] # List of association dfs by confidence
    rLift_list=[] # List of association dfs by lift
    cmpt_list=[] # List of potential complements dfs
    subs_list=[] # List of potential subs dfs

    df_input = data[(data['Point-of-Sale_ID']==pos)&(data['year']==(yyyy-2000))&(data['quarter']==q)][['Date','ProductName_ID']].copy()
    # Pivot the data - lines as orders and products as columns
    pt_input = pd.pivot_table(df_input, index='Date', columns='ProductName_ID', 
                        aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0)
    # Apriori
    freq_sets_input = apriori(pt_input, min_support=ms, use_colnames=True)
    # Association - by confidence
    rConf_input = association_rules(freq_sets_input, metric="confidence", min_threshold=cmt)
    rConf_input.sort_values(by='confidence', ascending=False, inplace=True)
    rConf_list.append(rConf_input)
    # Association - by lift
    rLift_input = association_rules(freq_sets_input, metric="lift", min_threshold=lmt)
    rLift_input.sort_values(by='lift', ascending=False, inplace=True)
    rLift_list.append(rLift_input)
    # High confidence and high lift - complementary products
    if cmptl_c != 99:
        cmpt_input = rConf_input[(rConf_input['confidence']>=cmptc)&(rConf_input['lift']>=cmptl)&(rConf_input['lift']<=cmptl_c)]
    else:
        cmpt_input = rConf_input[(rConf_input['confidence']>=cmptc)&(rConf_input['lift']>=cmptl)]
    cmpt_list.append(cmpt_input)
    # Subtitute products
    subs_input = association_rules(freq_sets_input, metric = 'lift',min_threshold=subsmt)
    subs_input.sort_values(by='lift', ascending=True, inplace=True)
    subs_list.append(subs_input)
    return rConf_list[0], rLift_list[0], cmpt_list[0], subs_list[0]

In [67]:
# This function is simplified version of the above, with limited capabilities.

def quarterly_association(pos,yyyy=2016,q=1, ms = 0.7): # minimum support below 0.7 seems to cause a serious stresss on our available computers. Admittedly, that is a very high value.
    mba_data = data[(data['Point-of-Sale_ID']==pos)&(data['year']==(yyyy-2000))&(data['quarter']==q)][['Date','ProductName_ID','Point-of-Sale_ID']].copy()
    #mba_data = mba_data[mba_data['ProductCategory_ID'].isin(cat_pos_q_top5['ProductCategory_ID'].unique())]
    mba_data = mba_data[['ProductName_ID','Date','Point-of-Sale_ID']].groupby(['Date', 'Point-of-Sale_ID'])['ProductName_ID'].apply(list)
    te = TransactionEncoder()
    te_ary = te.fit(mba_data).transform(mba_data)
    mba_data = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = apriori(mba_data, min_support= ms, use_colnames=True)
    rLift = association_rules(frequent_itemsets, metric = 'lift', min_threshold = 0.5)
    rLift.sort_values('lift', ascending = False, inplace = True)
    rConf = association_rules(frequent_itemsets, metric = 'confidence', min_threshold = 0.5)
    rConf.sort_values('confidence', ascending = False, inplace = True)
    return rLift, rConf

In [68]:
rLift_pos1, rConf_pos1 = quarterly_association(pos=1,yyyy=2019,q=3)

In [69]:
rLift_pos1.head()

antecedents              consequents  antecedent support  \
236508  (2609, 2346, 2412, 559)          (728, 226, 811)            0.739130   
236525          (728, 226, 811)  (2609, 2346, 2412, 559)            0.771739   
235991   (2412, 226, 2346, 559)         (728, 811, 1766)            0.760870   
236107    (728, 226, 811, 1766)        (2609, 2412, 559)            0.750000   
236547         (728, 2609, 811)   (2412, 226, 2346, 559)            0.750000   

        consequent support   support  confidence      lift  leverage  \
236508            0.771739  0.706522    0.955882  1.238608  0.136106   
236525            0.739130  0.706522    0.915493  1.238608  0.136106   
235991            0.750000  0.706522    0.928571  1.238095  0.135870   
236107            0.760870  0.706522    0.942029  1.238095  0.135870   
236547            0.760870  0.706522    0.942029  1.238095  0.135870   

        conviction  
236508    5.173913  
236525    3.086957  
235991    3.500000  
236107    4.125000  
236547    4.125000

In [70]:
rConf_pos1.head()

antecedents consequents  antecedent support  \
225231      (2408, 2346, 2609, 1846, 1277)       (811)            0.706522   
269093  (738, 1766, 811, 2412, 1277, 1855)       (226)            0.706522   
118297             (2609, 219, 2412, 1766)  (226, 811)            0.717391   
49386              (2609, 1277, 1846, 559)       (226)            0.771739   
184002      (1408, 1766, 2346, 1846, 1277)       (226)            0.750000   

        consequent support   support  confidence      lift  leverage  \
225231            0.923913  0.706522         1.0  1.082353  0.053757   
269093            0.967391  0.706522         1.0  1.033708  0.023039   
118297            0.913043  0.717391         1.0  1.095238  0.062382   
49386             0.967391  0.771739         1.0  1.033708  0.025165   
184002            0.967391  0.750000         1.0  1.033708  0.024457   

        conviction  
225231         inf  
269093         inf  
118297         inf  
49386          inf  
184002         inf

# 5. PoS Clusters <a class="anchor" id="5"></a>

## 5.1 Clustering by product sales


In [71]:
temp = data.groupby(['Point-of-Sale_ID', 'ProductName_ID']).size().unstack(fill_value=0)
temp.head()

ProductName_ID    1     2     3     4     5     6     7     8     9     10    \
Point-of-Sale_ID                                                               
1                    0     0     0     6     8    10     0     0     0     0   
2                    0     0     1    10     2    19     0     0     0     0   
3                    0     1     1     4     3     3     0     0     4     0   
4                    0     5    19    52    13   116     1     1     4     0   
5                    0     0     3     0     0     4     0     1     2     0   

ProductName_ID    ...  2841  2842  2843  2844  2845  2846  2847  2848  2849  \
Point-of-Sale_ID  ...                                                         
1                 ...   324     0     0     0   397   545   590    14    16   
2                 ...   337     0     0     0   215   248   951    69    22   
3                 ...   367     0     0     0   585   494  1140   228     0   
4                 ...   868     0     0     1   860   917  1279   101    24   
5                 ...   526     0     0     0   524   428  1109    41    38   

ProductName_ID    2850  
Point-of-Sale_ID        
1                   10  
2                    1  
3                   43  
4                   27  
5                    9  

[5 rows x 2820 columns]

In [72]:
temp = pd.read_csv('clustering_data.csv')
temp = temp[temp['Point-of-Sale_ID']!=96]

In [73]:
temp = temp.drop(columns=(['Point-of-Sale_ID','Quantity']))

In [74]:
temp.head()

1  2   3   4   5    6  7  8  9  10  ...  2841  2842  2843  2844  2845  \
0  0  0   0   6   8   10  0  0  0   0  ...   324     0     0     0   397   
1  0  0   1  10   2   19  0  0  0   0  ...   337     0     0     0   215   
2  0  1   1   4   3    3  0  0  4   0  ...   367     0     0     0   585   
3  0  5  19  52  13  116  1  1  4   0  ...   868     0     0     1   860   
4  0  0   3   0   0    4  0  1  2   0  ...   526     0     0     0   524   

   2846  2847  2848  2849  2850  
0   545   590    14    16    10  
1   248   951    69    22     1  
2   494  1140   228     0    43  
3   917  1279   101    24    27  
4   428  1109    41    38     9  

[5 rows x 2820 columns]

In [75]:
scores = []
for n in range(1, 11):
    model = KMeans(n_clusters = n, init = 'k-means++', random_state = 42)
    model.fit(temp)
    scores.append(model.inertia_)

fig = px.line(scores, title = 'Elbow Method', labels = {'index':'n_clusters', 'value':'inertia'})
fig.update_layout(showlegend = False)
fig.show()

In [76]:
model = KMeans(n_clusters=3, init = 'k-means++', random_state = 42)
len(model.fit_predict(temp))

409

In [77]:
pca = PCA(n_components=2)
temp = pca.fit_transform(temp)

In [78]:
fig = px.scatter(x = temp[:, 0], y = temp[:, 1], color = model.labels_, labels = {'x':'', 'y':''}, title = 'Points-of-Sale Clusters')
fig.show()

In [79]:
clustering_data = pd.read_csv('clustering_data.csv')
clustering_data = clustering_data[clustering_data['Point-of-Sale_ID']!=96]
clustering_data['clusters'] = model.fit_predict(temp)

In [80]:
clustering_data.drop(columns=(['Point-of-Sale_ID','Quantity']), inplace=True)

In [81]:
# top 10 products for cluster 0
clustering_data.groupby(['clusters']).mean().T.sort_values(by=[0],ascending=False).head(10)

clusters            0            1            2
356       2243.130841  1159.890625  1700.741379
253       1999.457944   772.640625  1190.413793
993       1692.850467  1146.539062  1448.160920
1369      1471.598131   538.648438  1021.959770
2609      1319.971963  1046.257812  1237.890805
481       1317.738318   981.328125  1190.977011
1277      1303.766355  1032.976562  1213.022989
1234      1292.084112   948.156250  1160.471264
1147      1283.252336   961.460938  1144.505747
847       1267.626168   944.226562  1157.545977

In [82]:
# top 10 products for cluster 1
clustering_data.groupby(['clusters']).mean().T.sort_values(by=[1],ascending=False).head(10)

clusters            0            1            2
356       2243.130841  1159.890625  1700.741379
993       1692.850467  1146.539062  1448.160920
2609      1319.971963  1046.257812  1237.890805
1277      1303.766355  1032.976562  1213.022989
481       1317.738318   981.328125  1190.977011
2802      1243.906542   972.312500  1148.195402
1147      1283.252336   961.460938  1144.505747
1234      1292.084112   948.156250  1160.471264
847       1267.626168   944.226562  1157.545977
567       1242.747664   903.406250  1105.339080

In [83]:
# top 10 products for cluster 2
clustering_data.groupby(['clusters']).mean().T.sort_values(by=[2],ascending=False).head(10)

clusters            0            1            2
356       2243.130841  1159.890625  1700.741379
993       1692.850467  1146.539062  1448.160920
2609      1319.971963  1046.257812  1237.890805
1277      1303.766355  1032.976562  1213.022989
481       1317.738318   981.328125  1190.977011
253       1999.457944   772.640625  1190.413793
1234      1292.084112   948.156250  1160.471264
847       1267.626168   944.226562  1157.545977
2802      1243.906542   972.312500  1148.195402
1147      1283.252336   961.460938  1144.505747

In [84]:
temp2 = data.groupby(['Point-of-Sale_ID', 'ProductCategory_ID']).size().unstack(fill_value=0)
temp2 = temp2.drop(temp2.index[96])
temp2.shape

(409, 178)

In [85]:
scores = []
for n in range(1, 11):
    model = KMeans(n_clusters = n, init = 'k-means++', random_state = 42)
    model.fit(temp2)
    scores.append(model.inertia_)

fig = px.line(scores, title = 'Elbow Method', labels = {'index':'n_clusters', 'value':'inertia'})
fig.update_layout(showlegend = False)
fig.show()

In [86]:
model = KMeans(n_clusters=3, init = 'k-means++', random_state = 42)
len(model.fit_predict(temp2))

409

In [87]:
pca = PCA(n_components=2)
temp2 = pca.fit_transform(temp2)

In [88]:
fig = px.scatter(x = temp2[:, 0], y = temp2[:, 1], color = model.labels_, labels = {'x':'', 'y':''}, title = 'Points-of-Sale Clusters')
fig.show()

In [89]:
clustering_data = pd.read_csv('clustering_data.csv')
clustering_data = clustering_data[clustering_data['Point-of-Sale_ID']!=96]
clustering_data['clusters'] = model.fit_predict(temp2)

In [90]:
clustering_data.drop(columns=(['Point-of-Sale_ID','Quantity']), inplace=True)

In [91]:
# top 10 products for cluster 0
clustering_data.groupby(['clusters']).mean().T.sort_values(by=[0],ascending=False).head(10)

clusters            0            1            2
356       2176.554545  1219.470149  1706.545455
253       1850.790909   864.388060  1215.496970
993       1685.300000  1148.925373  1457.775758
1369      1455.527273   595.000000   996.309091
2609      1318.081818  1051.119403  1240.678788
481       1303.836364  1003.977612  1187.169697
1277      1300.254545  1042.716418  1212.351515
1234      1280.845455   967.074627  1157.927273
847       1275.109091   934.574627  1166.151515
1147      1259.690909   989.731343  1141.387879

In [92]:
# top 10 products for cluster 0
clustering_data.groupby(['clusters']).mean().T.sort_values(by=[1],ascending=False).head(10)

clusters            0            1            2
356       2176.554545  1219.470149  1706.545455
993       1685.300000  1148.925373  1457.775758
2609      1318.081818  1051.119403  1240.678788
1277      1300.254545  1042.716418  1212.351515
481       1303.836364  1003.977612  1187.169697
1147      1259.690909   989.731343  1141.387879
2802      1240.300000   973.589552  1154.218182
1234      1280.845455   967.074627  1157.927273
847       1275.109091   934.574627  1166.151515
567       1228.518182   923.276119  1103.533333

In [93]:
# top 10 products for cluster 2
clustering_data.groupby(['clusters']).mean().T.sort_values(by=[2],ascending=False).head(10)

clusters            0            1            2
356       2176.554545  1219.470149  1706.545455
993       1685.300000  1148.925373  1457.775758
2609      1318.081818  1051.119403  1240.678788
253       1850.790909   864.388060  1215.496970
1277      1300.254545  1042.716418  1212.351515
481       1303.836364  1003.977612  1187.169697
847       1275.109091   934.574627  1166.151515
1234      1280.845455   967.074627  1157.927273
2802      1240.300000   973.589552  1154.218182
1147      1259.690909   989.731343  1141.387879

# 6. Forecast <a class="anchor" id="6"></a>

## 6.1 Units Product Forecast

In [94]:
lifetime_border = data['Date'].max() - datetime.timedelta(weeks = 6)
last_sale = data.groupby('ProductName_ID')['Date'].max()

relevant_products = last_sale[last_sale[last_sale >= lifetime_border].index].index

In [95]:
forecast_data = data[data['ProductName_ID'].isin(relevant_products)][['ProductName_ID','Date', 'Quantity']]
forecast_data = forecast_data.groupby(['ProductName_ID','Date']).sum()
forecast_data.reset_index(inplace=True)

In [96]:
#function to retrieve sales data per product
def prod_data_retriever(prod_id):
    prod_data = forecast_data[forecast_data['ProductName_ID'] == prod_id][['Date', 'Quantity']]
    prod_data.set_index('Date', inplace=True)
    prod_data = prod_data.groupby(pd.Grouper(freq = 'W')).sum()
    prod_data.sort_index(inplace = True)
    return prod_data

In [97]:
prod_data = prod_data_retriever(2609)

In [98]:
fig = px.line(prod_data, labels = {'Quantity':'Units Sold'}, width = 1200)
fig.update_layout(showlegend = False)

fig.show()

In [99]:
prod_data.drop(prod_data.index[0], inplace = True)
prod_data.drop(prod_data.index[-1], inplace = True)

In [100]:
fig = px.histogram(prod_data, title = 'Forecast Data Histogram')
fig.update_layout(showlegend = False)

In [101]:
#fits exponential models
def model_exp(i,j,b,train):
    if j!=None and i!=None:
        model = ExponentialSmoothing(np.asarray(train['Quantity']), trend=i, seasonal=j, seasonal_periods=52, damped=b)
    elif j!=None and i==None:
        model = ExponentialSmoothing(np.asarray(train['Quantity']), trend=i, seasonal=j, seasonal_periods=52)    
    elif j==None and i==None:
        model = ExponentialSmoothing(np.asarray(train['Quantity']), trend=i, seasonal=j)
    else:    
        model = ExponentialSmoothing(np.asarray(train['Quantity']), trend=i, seasonal=j, damped=b)
    return model

In [102]:
params = ['mul', 'add', None]  

In [103]:
#searchs for best parameters for exponential smoothing
def search(params,train,test):
    t = []
    s = []
    d = []
    r2_scores = []
    for i in params:
        for j in params:
            for b in [True, False]:
                model_expon = model_exp(i,j,b,train) 
                fit = model_expon.fit()
                pred = fit.forecast(test.shape[0])
                r2 = r2_score(test['Quantity'], pred)
                t.append(i)
                s.append(j)
                d.append(b)
                r2_scores.append(r2)
    df = pd.DataFrame({'t':t,'s':s, 'd':d, 'r2':r2_scores})
    return df

In [104]:
#forecasts for best model
def forecaster(mod,period,input_series,i,j,b):
    if mod == 'arima':
        smodel = pm.auto_arima(
                            input_series,
                            start_p = 1, start_q = 1,
                            max_q = 5, max_p = 5, m = 12,
                            trace = False)
        forecast = smodel.predict(n_periods = period, return_conf_int = False)
    elif mod == 'ses':
        ses_model = SimpleExpSmoothing(np.asarray(input_series))
        ses_fit = ses_model.fit(smoothing_level=.5)
        forecast = ses_fit.forecast(period)
    elif mod == 'holt':
        holt = Holt(np.asarray(input_series))
        holt_fit = holt.fit(smoothing_level=.3, smoothing_slope=.05)
        forecast = holt_fit.forecast(period)
    else:
        model_expon = model_exp(i,j,b,input_series) 
        fit = model_expon.fit()
        forecast = fit.forecast(period)
    return forecast

In [105]:
#take time series of the product sale data and number of weeks you want to do forecast for as inputs
def forecast(input_series,period):
    #divide data into test and train sets 20/80
    train = input_series[:round(input_series.shape[0]*0.80)]
    test = input_series[round(input_series.shape[0]*0.80):]
    #auto arima
    smodel = pm.auto_arima(
                            train,
                            start_p = 1, start_q = 1,
                            max_q = 5, max_p = 5, m = 12,
                            trace = False)
    #simple exponential smoothing
    ses_model = SimpleExpSmoothing(np.asarray(train['Quantity']))
    ses_fit = ses_model.fit(smoothing_level=.5)
    #holt
    holt = Holt(np.asarray(train['Quantity']))
    holt_fit = holt.fit(smoothing_level=.3, smoothing_slope=.05)
    #exponential smoothing
    exp = search(params,train,test)
    best = exp[exp['r2'] == exp['r2'].max()].values.tolist()[0]
    
    
    #make prediction    
    smodel_forecast = smodel.predict(n_periods = test.shape[0], return_conf_int = False)
    ses_forecast = ses_fit.forecast(test.shape[0])
    holt_forecast = holt_fit.forecast(test.shape[0])

    #r^2 values
    r2_scores = []
    r2_scores.append(r2_score(test['Quantity'], smodel_forecast))
    r2_scores.append(r2_score(test['Quantity'], ses_forecast))
    r2_scores.append(r2_score(test['Quantity'], holt_forecast))
    r2_scores.append(best[3])
    models = ['arima', 'ses', 'holt', 'exp']
    scores=pd.DataFrame({'model':models, 'r2':r2_scores})
    
    #choose the best model
    mod = scores[scores['r2'] == scores['r2'].max()].values.tolist()[0][0]
    r2_final = scores[scores['r2'] == scores['r2'].max()].values.tolist()[0][1]
    
    #forecast for future periods
    future_forecast = forecaster(mod,period,input_series,best[0],best[1],best[2])
    return future_forecast,r2_final,mod

In [106]:
top_5 = [1277,2609,481,993,1147]

In [107]:
r2_top5=[]
for i in top_5:
    prod_data = prod_data_retriever(i)
    prod_data.drop(prod_data.index[0], inplace = True)
    prod_data.drop(prod_data.index[-1], inplace = True)
    result = forecast(prod_data,6)
    r2_top5.append(result[1])

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_re

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_re

In [108]:
r2_top5

[0.7362797330794468,
 0.5231984544267212,
 0.08592384732260472,
 0.1541313916793614,
 0.42011764252375283]

In [109]:
print('average r2 score for top 5 products is: ', sum(r2_top5)/len(r2_top5))

average r2 score for top 5 products is:  0.3839302138063774


## 6.2 Units Product Forecast by Point-of-Sale 

In [110]:
forecast_data = data[data['ProductName_ID'].isin(relevant_products)][['Point-of-Sale_ID', 'ProductName_ID','Date', 'Quantity']]
forecast_data = forecast_data.groupby(['Point-of-Sale_ID','ProductName_ID','Date']).sum()
forecast_data.reset_index(inplace=True)

In [111]:
#function to retrieve sales data per product
def prod_data_retriever_pos(prod_id, pos_id):
    prod_data = forecast_data[(forecast_data['ProductName_ID'] == prod_id) & (forecast_data['Point-of-Sale_ID'] == pos_id)][['Date', 'Quantity']]
    prod_data.set_index('Date', inplace=True)
    prod_data = prod_data.groupby(pd.Grouper(freq = 'W')).sum()
    prod_data.sort_index(inplace = True)
    return prod_data

In [112]:
prod_data_retriever_pos(2609, 292)

Quantity
Date                
2016-01-03         8
2016-01-10        66
2016-01-17        88
2016-01-24        50
2016-01-31        83
...              ...
2019-10-06       125
2019-10-13       113
2019-10-20       122
2019-10-27       115
2019-11-03        85

[201 rows x 1 columns]

In [113]:
prod_data = prod_data_retriever_pos(2609,292)
prod_data.drop(prod_data.index[0], inplace = True)
prod_data.drop(prod_data.index[-1], inplace = True)

result = forecast(prod_data,6)

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:80: RuntimeWarning:

overflow encountered in matmul

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:80: RuntimeWarning:

overflow encountered in matmul

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:80: RuntimeWarning:

overflow encountered in matmul

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:80: RuntimeWarning:

overflow encountered in matmul



In [114]:
result

(array([121.62149059, 121.54810144, 121.4747123 , 121.40132316,
        121.32793402, 121.25454488]),
 0.018594785865458263,
 'holt')

In [115]:
r2_top5=[]
for i in top_5:
    prod_data = prod_data_retriever_pos(i, 292)
    prod_data.drop(prod_data.index[0], inplace = True)
    prod_data.drop(prod_data.index[-1], inplace = True)
    result = forecast(prod_data,6)
    r2_top5.append(result[1])

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:80: RuntimeWarning:

overflow encountered in matmul

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:80: RuntimeWarning:

overflow encountered in matmul

/home/doyun_s411/.local/lib/python3.6/site-p

In [116]:
r2_top5

[0.0007574092704245583,
 0.018594785865458263,
 0.8227440631758718,
 0.783516563950516,
 0.8004926168465738]

In [117]:
sum(r2_top5)/len(r2_top5)

0.4852210878217688

In [118]:
top_5

[1277, 2609, 481, 993, 1147]

# Utility

In [120]:
# Memory Usage check when needed
objects=[]
for name,obj in locals().items():
    objects.append([name,sys.getsizeof(obj)])
sorted(objects,key=lambda x: x[1],reverse=True)

[['forecast_data', 1960384188],
 ['data', 1705647084],
 ['rLift_pos1', 257115928],
 ['rConf_pos1', 257115928],
 ['cat_ms', 35545104],
 ['category', 21327200],
 ['clustering_data', 9231972],
 ['fam_ms', 7752188],
 ['family', 4544524],
 ['cat_ms_top', 1967124],
 ['fam_ms_top', 1901554],
 ['cat_pos_q_top5', 1442564],
 ['fam_pos_q_top5', 1376994],
 ['name_pos_q_top5', 1376994],
 ['_63', 392484],
 ['total_units_pos', 209976],
 ['last_sale', 127064],
 ['_71', 112864],
 ['_74', 112864],
 ['check', 21150],
 ['relevant_products', 15488],
 ['_70', 3960],
 ['a', 3376],
 ['_i66', 2794],
 ['_26', 2754],
 ['objects', 2536],
 ['obj', 2536],
 ['_69', 2424],
 ['_oh', 2280],
 ['Out', 2280],
 ['_112', 2034],
 ['prod_data', 2014],
 ['PCA', 2000],
 ['ExponentialSmoothing', 2000],
 ['SimpleExpSmoothing', 2000],
 ['Holt', 2000],
 ['_i105', 1921],
 ['_i2', 1566],
 ['_18', 1548],
 ['_19', 1548],
 ['KMeans', 1464],
 ['_i67', 1269],
 ['_9', 1174],
 ['_10', 1174],
 ['_ih', 1072],
 ['In', 1072],
 ['HTML', 1056],
 

In [121]:
# Total Memory Usage
import os, psutil
process = psutil.Process(os.getpid())
print(process.memory_info().rss/(1024**3))  # in gb

9.970977783203125
